**Welcome to Introduction to TensorFlow!**

In this lab, we will be understanding how to work with TensorFlow layers and functions.

We begin with importing the libraries required to continue further.

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

Then, we load the data into the memory.

In [3]:
data = pd.read_csv('data.csv')


If we look at the dataframe's properties, we see that there are many columns and rows.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In this example, we will start small. Let's select two columns to proceed with building a model.


This leads to building an equation of type:


y = f(x1,x2)

In [5]:
x = data[['radius_mean', 'perimeter_mean']]
y = data[['diagnosis']]

Let's have a look at what we have selected

In [6]:
x.head()

,radius_mean,perimeter_mean
0,17.99,122.80
1,20.57,132.90
2,19.69,130.00
3,11.42,77.58
4,20.29,135.10


Always check for null values

In [7]:
print(x.isna().sum())

radius_mean       0
perimeter_mean    0
dtype: int64


You can also view the general statistics of your dataset

In [8]:
x.describe()

,radius_mean,perimeter_mean
count,569.000000,569.000000
mean,14.127292,91.969033
std,3.524049,24.298981
min,6.981000,43.790000
25%,11.700000,75.170000
50%,13.370000,86.240000
75%,15.780000,104.100000
max,28.110000,188.500000


We had a look at our inputs. Now let's have a peek at the labels.

In [9]:
y.head()

,diagnosis
0,M
1,M
2,M
3,M
4,M


Normalization brings your dataset into a more managable range. We can apply z-scoring to normalize the dataset.

The formula for z-scoring is:

normalized_data = (data - mean) / standard_deviation

In [10]:
# Get the mean and standard deviation
stats = x.describe().T
mu, sigma = stats['mean'], stats['std']

In [11]:
# Normalize the data
x_norm = (x-mu)/sigma
x_norm.head()

,radius_mean,perimeter_mean
0,1.096100,1.268817
1,1.828212,1.684473
2,1.578499,1.565126
3,-0.768233,-0.592166
4,1.748758,1.775011


The inputs looks nice and clean now. But our labels are still tricky- they contain values like 'M' and 'B', but tensors work only on numbers.

We will now proceed to convert 'M' and 'B' into 1 and 0 so that we can build a classification model distributing the data into 1 and 0. We can then reverse map the predictions to infer 'M' or 'B'.

To encode our values from 'M' and 'B' into 1 and 0, we will use lambda functions, and then apply the function logic to every element of our dataframe.

In [12]:
rule = lambda val: 1 if val=='M' else 0

In [13]:
y=y['diagnosis'].apply(rule)

Let's check out the result of our encoding

In [14]:
y.tail()

,diagnosis
564,1
565,1
566,1
567,1
568,0


Now that it's all numbers, we can begin designing our first tensorflow network.

We begin with defining 'Hyperparameters'. Hyperparameters are values that you can fluctuate to alter model outputs. Parameters are from the dataset- every thing else that you can modify to alter the model results are hyperparamters!

In this example, we have 3 hyperparameters. Epochs tell us how long the model will train for or iterations. Batch size is the number of rows consumed in each step. Learning rate helps us reduce the loss over a period of time.

In [15]:
HP_epochs = 200
HP_batch_size = 16
HP_lr = 0.001

Then, we divide our dataset into training and scoring datasets. Training dataset will be used to build/train the model, while scoring dataset will be used to test the model.

In [16]:
xtrain,xtest,ytrain,ytest = train_test_split(x_norm,y,random_state=42)

Now, let's design our model. We are beginning with just 1 layer. This is a fully connected layer, represented as 'Dense' in TensorFlow. Using only one layer gives us the following model equation:


y = weight1 * x1 + weight2 * x2 + bias

In [17]:
model = tf.keras.Sequential([
    keras.layers.Dense(1)
])

The next step is to compile the model. Compiling specifies how the loss will be calculated, and optimized to yield a hopefully lower loss. This is where we use learning rate!

In [18]:
my_optimizer = tf.keras.optimizers.Adam(learning_rate=HP_lr)
my_error = 'mean_absolute_error'
model.compile(
    optimizer=my_optimizer,
    loss=my_error)

Notice how we could just use 'mean_absolute_error' as a string! TensorFlow allows us to use many such strings as predefined deep learning entities! Loss functions, optimization functions and metrics are such examples.

In [ ]:
# Do Not Uncomment- this is an alternate example
# FOR READING ONLY
# Note: instead of our own optimizer,
# we could have also defined our optimizer as
# my_optimizer = 'adam'

Now, we will train the model. Training is the process of fitting a curve/algebric equation through the dataset. This equation is then used to predict future inputs.

We are using the EPOCH hyperparameter here.

In [19]:
model.fit(xtrain, ytrain, epochs=HP_epochs)

Epoch 1/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5338   
Epoch 2/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5063 
Epoch 3/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4854 
Epoch 4/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4515 
Epoch 5/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4354 
Epoch 6/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4300 
Epoch 7/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4041 
Epoch 8/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3905 
Epoch 9/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3720 
Epoch 10/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3558 
Epoch 11/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3489 
Epoch 12/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3336 
Epoch 13/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3119 
Epoch 14/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2958 
Epoch 15/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - l

Let's have a look at the model we designed

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11 (48.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8 (36.00 B)

The trainable parameters can be calculated in the following manner:

We have two types of trainable parameters in this example- weights and bias.

Weights are calculated for each input and output. <br/>

Hence, total weights = inputs * output

Bias is calculated for each output. Total bias = output



Here, we have 2 inputs (x1 and x2), and 1 output (y).


Total parameters = weights + bias

Total parameters = input * output + output

In our case, this happens to be

Total parameters = 2 * 1 + 1

= 3

<br/>
Now let's get some predictions from our model, and have a look at them.

In [21]:
train_predictions = model.predict(xtrain)
test_predictions = model.predict(xtest)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [22]:
train_predictions[:5]

array([[0.15785019],
       [0.25020063],
       [0.18539312],
       [0.72263414],
       [1.0164073 ]], dtype=float32)

The model outputs don't look like 1 and 0- they are values calculated with the help of our trained model
<br/><br/>
y = weight1*x1 + weight2*x2 + bias

We will convert them now into 1 and 0, for the ease of calculation, and the ease of decoding to 'M' and 'B'.

In [23]:
rule = lambda val: 1 if val>=0.5 else 0

In [25]:
train_predictions = [ rule(prediction) for prediction in train_predictions]
test_predictions = [ rule(prediction) for prediction in test_predictions]

Let's have a look at the transformed labels

In [24]:
test_predictions[:5]

array([[0.13368517],
       [0.8350795 ],
       [0.4683416 ],
       [0.13525486],
       [0.02927271]], dtype=float32)

Now let's check how well our model performed. <br/><br/>
Accuracy is an easy equation- out of all guesses, how many did our model get right? A 100% accuracy is represented by 1, and 0% accuracy as 0.

How did our model perform on the dataset that it was trained on?

In [26]:
print(accuracy_score(train_predictions,ytrain))

0.8544600938967136


In [27]:
# or in %
print(str(accuracy_score(train_predictions,ytrain)*100) + "%")

85.44600938967136%


How did our model perform on unknown data?

In [28]:
accuracy_score(test_predictions,ytest)

0.8811188811188811

Are the values close enough? Are they too far away from each other? Is this overfitting? Is this underfitting? We will answer these questions slowly over the next few labs!

**Conclusion**

In this lab, we learnt the following:



*   Read the data and clean it
*   Break the data into inputs and labels
*   Normalize the inputs and encode the labels if required
*   Distribute the data into training and scoring datasets
*   Design the neural network and compile it
*   Train the model with training dataset
*   Test the model with scoring dataset
*   Use metrics (such as accuracy) to validate the dataset
<br/><br/>




